In [ ]:
from modules.engine import *
from modules.log_reader import read_qsym_log
import logging
import pandas as pd
from dataclasses import asdict

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
class NegationSolverSelectionStrategy(SolverSelectionStrategy):
    def get_solver(self, found_solver: MySolver | None, query: Query) -> MySolver:
        if found_solver is None:
            logging.info("No solver available, Creating a new one")
            solver = self.create_empty_solver()
            solver.upgrade(query.path[:-1], query.constraints[:-1])
            return solver

        solver = found_solver
        if found_solver.stack_path_len < len(query.path) - 1:
            logging.info("Creating a new solver for a longer path")
            solver = self.create_empty_solver()
            # Copying has much more overhead than adding again
            # solver.copy_from(found_solver)
            # solver.upgrade(query.path[solver.stack_path_len:-1],
            #                query.constraints[solver.constraint_count:-1])
            solver.upgrade(query.path[:-1], query.constraints[:-1])
        else:
            logging.info("Solver is appropriate")
        
        return solver

class ExactPathSolverSelectionStrategy(SolverSelectionStrategy):
    def get_solver(self, found_solver: MySolver | None, query: Query) -> MySolver:
        if found_solver is None:
            logging.info("No solver available, Creating a new one")
            solver = self.create_empty_solver()
            solver.upgrade(query.path, query.constraints)
            return solver

        solver = found_solver
        if found_solver.stack_path_len < len(query.path):
            logging.info("Creating a new solver for a longer path")
            solver = self.create_empty_solver()
            # Copying has much more overhead than adding again
            # solver.copy_from(found_solver)
            # solver.upgrade(query.path[solver.stack_path_len:],
                        #    query.constraints[solver.constraint_count:])
            solver.upgrade(query.path, query.constraints)
        else:
            logging.info("Solver is appropriate")
        
        return solver


In [ ]:
queries = read_qsym_log("./quicksort_big_export.log")
print("Largest query length:", max(len(q.constraints) for q in queries))
len(queries)

In [ ]:
pool = SolverPool(max_solvers=500)
pool.selection_strategy = BasicSolverSelectionStrategy()
# pool.selection_strategy = NegationSolverSelectionStrategy()
# pool.selection_strategy = ExactPathSolverSelectionStrategy()
pool.enable_solve_prefix_assertions(False)

In [ ]:
logging.getLogger().setLevel(level=logging.WARN)

for q in queries[:]:
    pool.solve(q)

logging.getLogger().setLevel(level=logging.INFO)

In [ ]:
print("Solver tree count:", len(pool._solver_trees))
print("Total alive solver count:", len(pool._solvers.items))
print("Most recently used solver:", list(pool._solvers.items.items())[-1][1]._solver)
print("Solver cache statistics:", pool._solvers.statistics)
solver_data = pd.DataFrame([asdict(stats) for stats in pool.statistics.solvers.values()])
solver_data = solver_data.drop(columns=["times"], inplace=False).join(solver_data.apply(lambda x: pd.Series(x["times"].values(), index=x["times"].keys()), axis=1))
pd.concat([solver_data.describe(), pd.Series(solver_data.sum(), name="sum").to_frame().transpose()])

In [ ]:
tree_data = pd.DataFrame([asdict(stats) for stats in pool.statistics.trees.values()])
tree_data = tree_data.drop(columns=["times"], inplace=False).join(tree_data.apply(lambda x: pd.Series(x["times"].values(), index=x["times"].keys()), axis=1))
tree_data.describe()